In [1]:
import pytest
import numpy as np
from pathlib import Path
import re
import pandas as pd
import networkx as nx
from cloudvolume import CloudVolume, Skeleton
from io import StringIO
from brainlit.utils.Neuron_trace import NeuronTrace
import os
import ipytest
ipytest.autoconfig()
ipytest.config.rewrite_asserts = True
___file___ = 'test_Neuron_trace.ipynb'

/Users/alishakodibagkar/opt/anaconda3/envs/brainlit/lib/python3.8/site-packages/python_jsonschema_objects/__init__.py:50: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


In [2]:
swc_path = "../../../tree_2.swc"
s3_path = "s3://open-neurodata/brainlit/brain1_segments"
seg_id = 11
mip = 2

seg_id_bad = 'hey'
mip_bad = 'hey'
read_offset_bad = 'hey'
path_bad_string = 'www.google.com'
path_bad_nonstring = 3


####################
### input checks ###
####################

def test_Neurontrace_bad_inputs():
    #test 'path' attribute must be a string
    with pytest.raises(TypeError):
        test_trace = NeuronTrace(path_bad_nonstring)
    
    #test 'path' attribute must be swc or skel path
    with pytest.raises(ValueError,match="Did not input 'swc' filepath or 'skel' url"):
        test_trace = NeuronTrace(path_bad_string)
    
    #test 'seg_id' attribute must be NoneType or int 
    with pytest.raises(TypeError):
        test_trace = NeuronTrace(s3_path,seg_id_bad,mip)
    
    #test 'mip' attribute must be NoneType or int
    with pytest.raises(TypeError):
        test_trace = NeuronTrace(s3_path,seg_id,mip_bad)
    
    # test both 'seg_id' and 'mip' attributes must be provided if one is provided
    with pytest.raises(ValueError,
        match="For 'swc' do not input mip or seg_id, and for 'skel', provide both mip and seg_id"):
        test_trace = NeuronTrace(s3_path,seg_id)
        
    #test 'read_offset' attribute must be bool
    with pytest.raises(TypeError):
        test_trace = NeuronTrace(swc_path,read_offset_bad)

def test_get_skel():
    #test 'origin' arg must either be type None or numpy.ndarray
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_skel(origin="hey")
    
    #test if 'origin' is type numpy.ndarray, it must be shape (3,1)
    with pytest.raises(ValueError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_skel(origin=np.asarray([0,1]))

def test_get_df_voxel():

    #test 'spacing' arg must be type numpy.ndarray
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_df_voxel(spacing='hey')
    
    #test if 'spacing' is type numpy.ndarray, it must be shape (3,1)
    with pytest.raises(ValueError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_df_voxel(spacing=np.asarray([0,1]))
    
    #test 'origin' arg must be type numpy.ndarray
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_df_voxel(spacing=np.asarray([0,1,2]),origin='hey')
    
    #test if 'origin' is type numpy.ndarray, it must be shape (3,1)
    with pytest.raises(ValueError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_df_voxel(spacing=np.asarray([0,1,2]),origin=np.asarray([0,1]))

def test_get_graph():

    #test 'spacing' arg must either be NoneType or numpy.ndarray
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_graph(spacing='hey')
    
    #test if 'spacing' is type numpy.ndarray, it must be shape (3,1)
    with pytest.raises(ValueError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_graph(spacing=np.asarray([0,1]))
    
    #test 'origin' arg must either be NoneType or numpy.ndarray
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_graph(spacing=np.asarray([0,1,2]),origin='hey')
    
    #test if 'origin' is type numpy.ndarray, it must be shape (3,1)
    with pytest.raises(ValueError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_graph(spacing=np.asarray([0,1,2]),origin=np.asarray([0,1]))

def test_get_paths():

    #test 'spacing' arg must either be NoneType or numpy.ndarray
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_paths(spacing='hey')
    
    #test if 'spacing' is type numpy.ndarray, it must be shape (3,1)
    with pytest.raises(ValueError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_paths(spacing=np.asarray([0,1]))
    
    #test 'origin' arg must either be NoneType or numpy.ndarray
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_paths(spacing=np.asarray([0,1,2]),origin='hey')
    
    #test if 'origin' is type numpy.ndarray, it must be shape (3,1)
    with pytest.raises(ValueError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_paths(spacing=np.asarray([0,1,2]),origin=np.asarray([0,1]))


def test_generate_df_subset():
    #test 'vox_in_img_list' arg must be type list
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.generate_df_subset(vox_in_img_list=2)
    
    #test 'subneuron_start' arg must either be NoneType or int
    with pytest.raises(TypeError):
        vox_in_img_list = [[72,53,128],[69,60,122],[69,63,115]]
        test_swc = NeuronTrace(swc_path)
        test_swc.generate_df_subset(vox_in_img_list,subneuron_start='hey',subneuron_end=3)
    
    #test 'subneuron_end' arg must either be NoneType or int
    with pytest.raises(TypeError):
        vox_in_img_list = [[72,53,128],[69,60,122],[69,63,115],[74,63,108],[155,162,57]]
        test_swc = NeuronTrace(swc_path)
        test_swc.generate_df_subset(vox_in_img_list,subneuron_start=3,subneuron_end='hey')
    
    #test both 'subneuron_start' or 'subneuron_end must' be specified, if only 1 is
    with pytest.raises(ValueError,match="Provide both starting and ending vertices to use for the subneuron"):
        vox_in_img_list = [[72,53,128],[69,60,122],[69,63,115],[74,63,108],[155,162,57]]
        test_swc = NeuronTrace(swc_path)
        test_swc.generate_df_subset(vox_in_img_list,subneuron_start=3)

def test_get_bfs_subgraph():
    #test 'node_id' arg must be type int
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_bfs_subgraph(node_id='hey',depth=2)
    
    #test 'depth' arg must be type int
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_bfs_subgraph(node_id=11,depth='hey')
    
    #test 'df' arg must be either NoneType or pandas.DataFrame
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_bfs_subgraph(node_id=11,depth=2,df='hey')
    
    #test 'spacing' arg must either be NoneType or numpy.ndarray
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_bfs_subgraph(node_id=11,depth=2,spacing=1)
    
    #test if 'spacing' is type numpy.ndarray, it must be shape (3,1)
    with pytest.raises(ValueError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_bfs_subgraph(node_id=11,depth=2,spacing=np.asarray([1,2]))
    
    #test 'origin' arg must either be NoneType or numpy.ndarray
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_bfs_subgraph(node_id=11,depth=2,spacing=np.asarray([1,2,3]),origin=1)
    
    #test if 'origin' is type numpy.ndarray, it must be shape (3,1)
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_bfs_subgraph(node_id=11,depth=2,spacing=np.asarray([1,2,3]),origin=np.asarray[1,2])

def test_get_sub_neuron():

    #test 'bounding_box' arg is a tuple or list
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_sub_neuron(bounding_box=1)
    
    #test if 'bounding_box' arg is tuple or list, length is 2
    with pytest.raises(ValueError,match='Bounding box must be length 2'):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_sub_neuron(bounding_box=[[1,2,4],[1,2,3],[4,5,6]])
    
    #test 'spacing' arg must either be NoneType or numpy.ndarray
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_sub_neuron(bounding_box=[[1,2,4],[1,2,3]],spacing='hey')
    
    #test if 'spacing' is type numpy.ndarray, it must be shape (3,1)
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_sub_neuron(bounding_box=[[1,2,4],[1,2,3]],spacing=[1,2])
    
    #test 'origin' arg must either be NoneType or numpy.ndarray 
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_sub_neuron(bounding_box=[[1,2,4],[1,2,3]],spacing=[1,2,3],origin=1)
    
    #test if 'origin' is type numpy.ndarray, it must be shape (3,1)
    with pytest.raises(TypeError):
        test_swc = NeuronTrace(swc_path)
        test_swc.get_sub_neuron(bounding_box=[[1,2,4],[1,2,3]],spacing=[1,2,3],origin=[1,2])

In [3]:
ipytest.run()

........                                                                 [100%]
=============================== warnings summary ===============================
../../../../../../opt/anaconda3/envs/brainlit/lib/python3.8/site-packages/_pytest/config/__init__.py:1114
  /Users/alishakodibagkar/opt/anaconda3/envs/brainlit/lib/python3.8/site-packages/_pytest/config/__init__.py:1114: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: napari_plugin_engine
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/warnings.html
8 passed, 1 warning in 0.09s
